In [1]:
from tkinter import *
from tkinter.messagebox import askokcancel
import server


class Server_GUI:
    def __init__(self, master):
        self.services = None
        self.master = master

        self.master.title("Server")
        self.master.geometry('200x200')

        self.btn_start = Button(self.master, text = "Start Server", width = 20, height = 5, command = self.start)
        self.btn_start.place(relx = 0.5, rely = 0.5, anchor = CENTER)

        self.master.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start(self):
        if self.services == None:
            try:
                self.services = server.ServerServices()
                self.services.startServices()

                self.btn_start.config(text = "Stop Server")
            except:
                print('Cannot create server')
                self.services = None
        else:
            self.services.stopServices()
            self.services = None

            self.btn_start.config(text = "Start Server")

    def on_closing(self):
        if self.services != None:
            if askokcancel("Quit", "Server is running.\nDo you want to quit?"):
                self.services.stopServices()
            else:
                return
            
        self.master.destroy()

            

window_server = Tk()
Server_GUI(window_server)
window_server.mainloop()

START SERVER
Port: 1234
('127.0.0.1', 53016) 	START CONNECTION
('127.0.0.1', 53016) 	Receive: processlist
('127.0.0.1', 53016) 	SEND PROCESS LIST
('127.0.0.1', 53016) 	Sent dump data. Size: 6329
('127.0.0.1', 53016) 	Receive: processlist
('127.0.0.1', 53016) 	SEND PROCESS LIST
('127.0.0.1', 53016) 	Sent dump data. Size: 6357
('127.0.0.1', 53016) 	Receive: killprocess
('127.0.0.1', 53016) 	KILL PROCESS 13564
('127.0.0.1', 53016) 	Receive: killprocess
('127.0.0.1', 53016) 	KILL PROCESS 13564
('127.0.0.1', 53016) 		Err: Unable to kill 13564
('127.0.0.1', 53016) 	Client have unexpected error. Exit thread
STOP SERVER
('127.0.0.1', 53016) 	CLOSE CONNECTION
Server stopped. Exit thread


In [ ]:
from winreg import *
def getHKEY(name):
    if name == "HKEY_CLASSES_ROOT":
        return HKEY_CLASSES_ROOT
    elif name == "HKEY_CURRENT_USER":
        return HKEY_CURRENT_USER
    elif name == "HKEY_LOCAL_MACHINE":
        return HKEY_LOCAL_MACHINE
    elif name == "HKEY_USERS":
        return HKEY_USERS
    elif name == 'HKEY_PERFORMANCE_DATA':
        return HKEY_PERFORMANCE_DATA
    elif name == 'HKEY_CURRENT_CONFIG':
        return HKEY_CURRENT_CONFIG
    elif name == 'HKEY_DYN_DATA':
        return HKEY_DYN_DATA

def getType(type):
    if type == 'String':
        return REG_SZ
    elif type == 'Binary':
        return REG_BINARY
    elif type == 'DWORD':
        return REG_DWORD
    elif type == 'QWORD':
        return REG_QWORD
    elif type == 'Multi-String':
        return REG_MULTI_SZ
    elif type == 'Expandable String':
        return REG_EXPAND_SZ

def createKey(key):
    hkey, key = key.split('\\', 1)
    return CreateKeyEx(getHKEY(hkey), key)

def deleteKey(key):
    hkey, key = key.split('\\', 1)
    DeleteKey(getHKEY(hkey), key)

def getValue(key, valueName):
    return QueryValueEx(createKey(key), valueName)

def setValue(key, valueName, value, type):
    return SetValueEx(createKey(key), valueName, 0, getType(type), value)

def deleteValue(key, valueName):
    DeleteValue(createKey(key), valueName)



In [ ]:
from _winreg import *

def createKey(key):
    hkey, key = key.split('\\', 1)
    return CreateKeyEx(getHKEY(hkey), key)

def deleteKey(key):
    hkey, key = key.split('\\', 1)
    DeleteKey(getHKEY(hkey), key)

def getValue(key, valueName):
    return QueryValueEx(createKey(key), valueName)

def setValue(key, valueName, value, type):
    return SetValueEx(createKey(key), valueName, 0, getType(type), value)

def deleteValue(key, valueName):
    DeleteValue(createKey(key), valueName)

SetValueEx(createKey(r'HKEY_LOCAL_MACHINE\SOFTWARE\Adobe\test1\test2'), 'S2', 0, _winreg.REG_MULTI_SZ, ["Car", "Bus", "Train"])

In [ ]:
setValue(r'HKEY_LOCAL_MACHINE\SOFTWARE\Adobe\test1\test2', 'test 1', b'ahihi', 'Binary')

In [ ]:
createKey(r'HKEY_LOCAL_MACHINE\SOFTWARE\Adobe\test1\test2')

In [ ]:
deleteKey(r'HKEY_LOCAL_MACHINE\SOFTWARE\Adobe\test1\test2')

In [ ]:
deleteValue(r'HKEY_LOCAL_MACHINE\SOFTWARE\Adobe\test1\test2', 'a')

In [ ]:
getValue(r'HKEY_LOCAL_MACHINE\SOFTWARE\Adobe\test1\test2', r'a')

In [5]:
import os
print(os.kill(15200, 9))

None


In [ ]:

import ctypes
import subprocess
import sys
import os

if os.name == 'nt':
	if not ctypes.windll.shell32.IsUserAnAdmin():
		print('\nThis script requires to be ran with admin privileges\n')
		sys.exit()

if sys.version[0] == '2':
	input = raw_input
	import _winreg as wreg
else:
	import winreg as wreg

#create key
key = wreg.CreateKey(wreg.HKEY_LOCAL_MACHINE, "Software\\TestCompany\\TestProject")
# Create new subkey
wreg.SetValue(key, 'NewSubkey', wreg.REG_SZ, 'testsubkey')
print(wreg.QueryValue(key, 'NewSubKey'))
# prints 'testsubkey'
# Create new value
wreg.SetValueEx(key, 'ValueName', 0, wreg.REG_SZ, 'testvalue')
print(wreg.QueryValueEx(key,'ValueName'))
# prints (u'testvalue', 1)
key.Close()


#read value
key = wreg.OpenKey(wreg.HKEY_LOCAL_MACHINE, "Software\\TestCompany\\TestProject",0,wreg.KEY_ALL_ACCESS)
wreg.SetValue(key, 'NewSubkey', wreg.REG_SZ, 'subkey_changed')
print(wreg.QueryValue(key, 'NewSubkey'))
# prints 'subkey_changed'
wreg.SetValueEx(key, 'ValueName', 0, wreg.REG_SZ, 'value_changed')
print(wreg.QueryValueEx(key, 'ValueName'))
key.Close()


#loop keys
keyVal = r"Software"
aKey = wreg.OpenKey(wreg.HKEY_LOCAL_MACHINE, keyVal, 0, wreg.KEY_ALL_ACCESS)
try:
    i = 0;
    while True:
        asubkey = wreg.EnumKey(aKey, i)
        print(asubkey)
        i += 1
except WindowsError:
    pass

input('Press Enter')

In [ ]:
import subprocess

lines = subprocess.Popen("wmic process get description, processid, threadcount", shell=True, stdout=subprocess.PIPE)

firstLine = lines.stdout.readline().decode().rstrip()
headers = ['Description', 'ProcessId', 'ThreadCount']
align = []
for header in headers:
    align.append(firstLine.find(header))
align.append(len(firstLine) + 1)

print(align)

result = []
for l in lines.stdout:
    line = l.decode().rstrip()
    row = []
    for i in range(len(align) - 1):
        row.append(line[align[i] : align[i + 1] - 1].rstrip())
    print(row)
    result.append(row)


data.stdout.close()

In [ ]:
# vấn đề với hàm cmd
def getCommand(buff):
    command = (buff.recv_until(DELIM).decode())
    print(f'COMMAND {command}')

    returnVal = os.system(command)
    sendDump(buff, returnVal)

In [4]:
import subprocess
import win32api
import win32process


subprocess.Popen('C:\\Program Files (x86)\\Microsoft Office\\root\\Office16\\MSACESS.EXE')

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
import subprocess
import win32api
import win32process


#subprocess.Popen('notepad')
#win32api.ShellExecute(None, None, "notepad.exe", None, None, 5)
win32process.CreateProcess(None, "C:\\Program Files (x86)\\Microsoft Office\\root\\Office16\\MSACCESS.EXE", None, None, 8, win32process.NORMAL_PRIORITY_CLASS, None, None, win32process.STARTUPINFO())

In [ ]:
import subprocess
cmd = 'powershell "gps | where {$_.MainWindowTitle } | select Description'
proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
for line in proc.stdout:
     if line.rstrip():
        # only print lines that are not empty
        # decode() is necessary to get rid of the binary string (b')
        # rstrip() to remove \r\n
        print(line.decode().rstrip())